## Parameter Estimation

In [ ]:
from progpy.models import ThrownObject
import matplotlib.pyplot as plt

times = [0, 1, 2, 3, 4, 5, 6, 7]
inputs = [{}]*8
outputs = [
    {'x': 1.83},
    {'x': 36.5091999066245},
    {'x': 60.05364349596605},
    {'x': 73.23733081022635},
    {'x': 76.47528104941956},
    {'x': 69.9146810161441},
    {'x': 53.74272753819968},
    {'x': 28.39355725512131},
]

m = ThrownObject(thrower_height=20, throwing_speed=3.1, g=15)

keys = ['thrower_height', 'throwing_speed', 'g']

print('Model configuration before')

for key in keys:
    print("-", key, m[key])

print('\nError: ', m.calc_error(times, inputs, outputs, dt=0.1))

In [ ]:
# Plot real data
pos = [state['x'] for state in outputs]
plt.plot(times, pos, color="black", linestyle="--")

# Plot simulation
simulated_results = m.simulate_to(times[-1], save_freq=1)
pos = [state['x'] for state in simulated_results.outputs]
plt.plot(times, pos, color="blue", linestyle="--")

plt.show()

In [ ]:
m.estimate_params(times = times, inputs = inputs, outputs = outputs, keys = keys, dt=0.1)

print('\nOptimized configuration')

for key in keys:
    print("-", key, m[key])
    
print('\nError: ', m.calc_error(times, inputs, outputs, dt=0.1))

In [ ]:
# Plot real data
pos = [state['x'] for state in outputs]
plt.plot(times, pos, color="black", linestyle="--")

# Plot simulation
simulated_results = m.simulate_to(times[-1], save_freq=1)
pos = [state['x'] for state in simulated_results.outputs]
plt.plot(times, pos, color="blue", linestyle="--")

plt.show()

## Noise Estimation

For a single noise value, have multiple simulations for that and that will show the distribution of possible values at that noise level
- Note that each noise value samples from a distribution already
- Use MonteCarlo from monte_carlo.py
- In 2024PHMTutorial, see monte_carlo example

In [ ]:
from progpy.models import ThrownObject
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize

# Plot real data
times = [0, 1, 2, 3, 4, 5, 6, 7]
inputs = [{}]*8
outputs = [
    {'x': 1.83},
    {'x': 36.5091999066245},
    {'x': 60.05364349596605},
    {'x': 73.23733081022635},
    {'x': 76.47528104941956},
    {'x': 69.9146810161441},
    {'x': 53.74272753819968},
    {'x': 28.39355725512131},
]
pos = [state['x'] for state in outputs]
plt.plot(times, pos, color="black", linestyle="--", label="Real data")

# Plot model with no noise
m = ThrownObject(process_noise=0, measurement_noise=0)
simulated_results = m.simulate_to(times[-1], save_freq=1)
pos = [state['x'] for state in simulated_results.outputs]
plt.plot(times, pos, color="blue", linestyle="--", label="No noise simulation")

# Plot monte carlo simulations with random noise values
def estimate_noise(m, times, inputs, outputs):
    min_error = float('inf')   

    for _ in range(15):
        m.parameters['process_noise'] = abs(np.random.normal(0, 6))
        m.parameters['measurement_noise'] = abs(np.random.normal(0, 6))
        simulated_results = m.simulate_to(times[-1], save_freq=1)

        pos = [state['x'] for state in simulated_results.outputs]
        data_pos = [state['x'] for state in outputs]
        
        error = np.mean((np.array(pos) - np.array(data_pos))**2)
        print('Error:', error)
        
        plt.plot(simulated_results.times, pos)

estimate_noise(m, times, inputs, outputs)
plt.legend()
plt.show()

In [ ]:
from progpy.models import ThrownObject
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize

# Plot real data
times = [0, 1, 2, 3, 4, 5, 6, 7]
inputs = [{}]*8
outputs = [
    {'x': 1.83},
    {'x': 36.5091999066245},
    {'x': 60.05364349596605},
    {'x': 73.23733081022635},
    {'x': 76.47528104941956},
    {'x': 69.9146810161441},
    {'x': 53.74272753819968},
    {'x': 28.39355725512131},
]
pos = [state['x'] for state in outputs]
plt.plot(times, pos, color="black", linestyle="--", label="Real data")

# Plot model with no noise
m = ThrownObject(process_noise=0, measurement_noise=0)
simulated_results = m.simulate_to(times[-1], save_freq=1)
pos = [state['x'] for state in simulated_results.outputs]
plt.plot(times, pos, color="blue", linestyle="--", label="No noise simulation")

# Plot monte carlo simulations with random noise values
def estimate_noise(m, times, inputs, outputs):
    min_error = float('inf')   

    for _ in range(100):
        m.parameters['process_noise'] = abs(np.random.normal(0, 6))
        m.parameters['measurement_noise'] = abs(np.random.normal(0, 6))
        simulated_results = m.simulate_to(times[-1], save_freq=1)

        pos = [state['x'] for state in simulated_results.outputs]
        data_pos = [state['x'] for state in outputs]
        
        error = np.mean((np.array(pos) - np.array(data_pos))**2)

        if error < min_error:
            min_error = error
            min_process_noise = m.parameters['process_noise']
            min_measurement_noise = m.parameters['measurement_noise']
            min_pos = pos
        
    print('Error:', min_error)
    print('Process noise:', min_process_noise)
    print('Measurement noise:', min_measurement_noise)
    plt.plot(simulated_results.times, min_pos, label="Noise simulation")

estimate_noise(m, times, inputs, outputs)
plt.legend()
plt.show()